In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

# Definie initial conditions:
r = 0
k = 0
m = 0
delta = 0
beta = 0
gamma = 0
C_1 = 0
C_2 = 0
C_3 = 0
C_4 = 0

h = 0.1  # Step size
n_steps = int(1 / h)  # Number of steps for x in [0, 1]

# Initial values
t0 = 0.0
y0 = 0.0

# Define ODEs:
def A(t,y):
    return C_1 * np.exp((r - m - m) * t - (k* C_2) * np.exp(-delta*t)/(delta))

def B(t,y):
    return C_2 * np.exp(-delta*t)

def C(t,y):
    integrand = k * A(t,y) * B(t,y) * np.exp(beta + gamma)*t
    C_int = odeint(integrand, 0, t, args=(A(t,y), B(t,y), beta, gamma, k))
    return np.exp(-(beta + gamma)*t) + C_int + C_3 * np.exp(-(beta + gamma) * t)

def Ap(t,y):
    integrand = np.exp((r + k)*t) * (m*A(t,y) + gamma*C(t,y))
    Ap_int = odeint(integrand, 0, t, args=(A(t,y), C(t,y), gamma, r, m, k))
    return np.exp(-(r+k)*t) * Ap_int + C_4 * np.exp(-(r+k)*t)


# iterative methods


# plot results